In [1]:
MODELS = [
    
]

ITERATIONS_TO_TEST = [
    1
]

NUM_PROCS = 4
GAMES_PER_CONFIG = 10

In [2]:
import os
os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 
os.environ["OMP_NUM_THREADS"] = "1" 
import torch
torch.set_num_threads(1)
from train import load_from_checkpoint, test_network
import torch.multiprocessing as mp
from resnet import ResNet2Heads
from copy import deepcopy
from utils import input_to_tensor_scalar
import time
import warnings
import csv

In [3]:
model_results = dict()

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for model_path in MODELS:
        _, model, _, hyperparameters, _, _, _ = load_from_checkpoint(model_path, ResNet2Heads, load_replay_memory=False)
        csv_filename = f'benchmark_results_{model_path.split("/")[-1][:-3]}.csv'
        trials = 0
        def write_trial_results(results):
            _, moves, high_square, score = results
            with open(csv_filename, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([moves, high_square, score, iter_qty])
            global trials
            trials += 1
            print(f'score={score}, moves={moves}, high_square={high_square}, trial_num={trials}')

        for iter_qty in ITERATIONS_TO_TEST:
            trials = 0
            print(model_path, iter_qty)
            hyperparameters.num_mcts_test_evals = iter_qty
            start_time = time.time()
            with mp.Pool(NUM_PROCS) as pool:
                results = []
                for _ in range(GAMES_PER_CONFIG):
                    results.append(pool.apply_async(test_network, (model, hyperparameters, input_to_tensor_scalar, ), callback=write_trial_results, error_callback=print))
                for r in results:
                    r.wait()
            print(f'took {time.time() - start_time}')
